# This is an example of doing the geocoding we did with OpenRefine but using Python to call the same service

In [ ]:
#add your app_id here
APP_ID = '48eab85b'
#add your app_key here
APP_KEY = 'fa31219b4e426ff96a8ded2905e6ae3a'

In [ ]:
#We have to tell Python where to find the input file, giving the full path to the file as the first argument
#and then whether it's reading the file ('r') or writing to the file ('w'). We could also append to a file ('a')
#for more information, see http://www.tutorialspoint.com/python/python_files_io.htm
inputFile = open('job0714.csv','rU') #change to match the file location
header = next(inputFile) #take the next line of input and assign it to a variable

In [ ]:
header #the value of the variable header

In [ ]:
import csv #package for dealing with csv files

In [ ]:
#We use the csv.reader function to read in the columns properly, especially the ones that have commas in between
#double quotation marks.  This is a built in package to Python, but we still have to link to it with the import statement above
#for more information, see https://docs.python.org/2/library/csv.html
inputReader = csv.reader(inputFile) 

In [ ]:
for i in range(10):
    print next(inputReader)

In [ ]:
test_line = next(inputReader) #the next function returns a line and I'm assigning that line to a variable. 
# This helps me test the input from the text.
test_line

In [ ]:
test_line[7]

In [ ]:
#because we've already iterated through some of the file contents, it's best to re-open the file and start from the beginning
inputFile = open('job0714.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [ ]:
#this time, we're just printing out the BIN
for line in inputReader:
    print line[7]
inputFile.close() #close the open input file

In [ ]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('job0714.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [ ]:
#This time, we're taking the BIN and inserting it into the API string
#I can check the string by clicking the link and it should provide me the response I'm looking for
#this allows me to test that I've constructed the string properly
for line in inputReader: 
    print('https://api.cityofnewyork.us/geoclient/v1/bin.json?bin=' + line[7] +'&app_id=' + APP_ID + '&app_key=' + APP_KEY)

inputFile.close() #close the open input file

In [ ]:
#because we've already iterated through and closed the file, we have to re-open the file
inputFile = open('job0714.csv','rU') 
header = next(inputFile) #take the next line and assign it to a variable
inputReader = csv.reader(inputFile) 

In [ ]:
#This time, I want to process the response I get. I'm going to get the response and write the output to a file
#Here I'm opening the file I'm going to write to. If the file already exists, it overwrite the file.
outputFile = open('job0714_output.csv','w') #opens the file
outputFile.write('job_number,bin_num,longitude,latitude\n') #writes the first line of my CSV file. NOTE: '\n' writes a new line

In [ ]:
import requests #package for executing web requests ('HTTP for Humans')
#for more information, check the docs at http://docs.python-requests.org/en/latest/

In [ ]:
#This is a test call using BIN 1013618 to see what the rerequests.get
r = requests.get('https://api.cityofnewyork.us/geoclient//v1/bin.json?bin=1013618&app_id=' + APP_ID + '&app_key=' + APP_KEY)

In [ ]:
r.json()['bin']['latitudeInternalLabel']

In [ ]:
#this is an example of the value from the requests.get() function
#the 'u' before the string indicates it's in Unicode format. 
#for more information on Unicode characters, see http://en.wikipedia.org/wiki/Unicode_symbols
#I use this test response to know which fields to use in my for loop later on
r.json()['bin']

In [ ]:
#This time through, I'm executing the web request, I'm getting the reponse, and then writing it to the output
for line in inputReader:
    #the web request, using the get() function from the package requests and assigning it to the variable r
    r = requests.get('https://api.cityofnewyork.us/geoclient//v1/bin.json?bin=' + line[7] +'&app_id=' + APP_ID + '&app_key=' + APP_KEY)
    job_number = line[0] #I'm getting the job number from the line (at position 0) and assigning it to the variable job_number
    bin_num = line[7] #I'm getting the BIN from the line (at position 7) and assigning it to the variable bin_num

    #My default value is 'Null' for longitude. 0 is a legitimate value and when I map these, I don't want anything to plot to 0,0
    longitude = 'Null' 
    latitude = 'Null'

    #The r variable has a function called json() that takes the allows me to interact with the JSON document from the API
    #I'm going to parse the response and extract the longitude and latitude from the response
    
    #longitude
    if 'longitudeInternalLabel' in r.json()['bin'].keys(): #I first check that there is a longitude present
        #if it's present, I assign the value to the variable "longitude". Otherwise, the variable longitude has the value 'Null'
        longitude = r.json()['bin']['longitudeInternalLabel'] 
    #latitude
    if 'latitudeInternalLabel' in r.json()['bin'].keys(): #I then check that there is a latitude present
        #if it's present, I assign the value to the variable "latitude". Otherwise, the variable latitude has the value 'Null'
        latitude = r.json()['bin']['latitudeInternalLabel']
    #I then write the result to the output file, stringing together the variables
    #because the latitude and longitude are decimal numbers, I have to convert them to strings using the str() function
    #in order to combine them together with the rest of the variables. 
    #Note the commas in the string and the '\n' new line character at the end of the string
    outputFile.write(job_number + ',' + bin_num + ',' + str(longitude) + ',' + str(latitude) + '\n')

inputFile.close() #close the input file
outputFile.close() #close the open output file. 
#This is important because if you don't close it, it's possible the last bit of data won't get written to the file before the 
#program closes.